# Yield Prediction: Ahneman Buchwald-Hartwig Dataset

**Goal:** Predict reaction yield using ML

**Approach:** Following ORD creator's method:
1. Convert reactions to DataFrame using `messages_to_dataframe()`
2. One-hot encode categorical inputs (aryl halide, amine, ligand, base, additive)
3. Train neural network to predict yield

**Why this IS real ML:** Same substrates + different conditions → different yields

---

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from ord_schema import message_helpers, validations
from ord_schema.proto import dataset_pb2, reaction_pb2

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import tensorflow as tf
from tensorflow import keras

sns.set_style("whitegrid")
np.random.seed(42)
tf.random.set_seed(42)

## 1. Load Dataset

In [ ]:
# Load Ahneman dataset
dataset_path = "ord-data/data/46/ord_dataset-46ff9a32d9e04016b9380b1b1ef949c3.pb.gz"
data = message_helpers.load_message(dataset_path, dataset_pb2.Dataset)

print(f"Dataset: {data.name}")
print(f"Description: {data.description}")
print(f"Reactions: {len(data.reactions)}")

In [ ]:
# Convert to DataFrame using ORD helper
df = message_helpers.messages_to_dataframe(data.reactions, drop_constant_columns=True)

print(f"DataFrame shape: {df.shape}")
print(f"\nColumns ({len(df.columns)}):")
for col in df.columns:
    print(f"  {col}")

In [ ]:
# Preview the data
df.head()

## 2. Select Modeling Columns

We need to identify:
- Input columns (SMILES for reactants, catalysts, etc.)
- Output column (yield)

In [ ]:
# Find columns containing SMILES identifiers and yield
print("Looking for input and output columns...\n")

smiles_cols = [col for col in df.columns if 'identifiers' in col and 'SMILES' in str(df[col].iloc[0])]
yield_cols = [col for col in df.columns if 'percentage' in col.lower() or 'yield' in col.lower()]

print("Potential SMILES columns:")
for col in smiles_cols[:10]:
    print(f"  {col}")

print(f"\nPotential yield columns:")
for col in yield_cols:
    print(f"  {col}")

In [ ]:
# Let's explore the input structure more carefully
# Find columns with 'inputs' to understand the reaction components

input_cols = [col for col in df.columns if col.startswith('inputs')]
print(f"Input columns ({len(input_cols)}):")
for col in sorted(set([c.split('.')[0] + '.' + c.split('.')[1] if '.' in c else c.split('[')[0] + '[' + c.split('[')[1].split(']')[0] + ']' for c in input_cols])):
    print(f"  {col}")

In [ ]:
# Explore one reaction to understand structure
rxn = data.reactions[0]

print("=== REACTION INPUTS ===")
for key, inp in rxn.inputs.items():
    print(f"\n{key}:")
    for comp in inp.components:
        role = reaction_pb2.ReactionRole.ReactionRoleType.Name(comp.reaction_role)
        for ident in comp.identifiers:
            ident_type = reaction_pb2.CompoundIdentifier.CompoundIdentifierType.Name(ident.type)
            if ident_type == "SMILES":
                print(f"  {role}: {ident.value[:60]}")

print("\n=== YIELD ===")
for outcome in rxn.outcomes:
    for product in outcome.products:
        for meas in product.measurements:
            if meas.type == reaction_pb2.ProductMeasurement.ProductMeasurementType.YIELD:
                print(f"  Yield: {meas.percentage.value}%")

In [ ]:
# Extract data manually for cleaner control
def extract_reaction_data(reaction):
    """Extract key components and yield from a reaction."""
    data = {}
    
    # Extract inputs by key name
    for key, inp in reaction.inputs.items():
        for comp in inp.components:
            for ident in comp.identifiers:
                if reaction_pb2.CompoundIdentifier.CompoundIdentifierType.Name(ident.type) == "SMILES":
                    # Use the input key as the column name
                    clean_key = key.replace(' ', '_').lower()
                    data[clean_key] = ident.value
                    break
    
    # Extract yield
    data['yield'] = None
    for outcome in reaction.outcomes:
        for product in outcome.products:
            for meas in product.measurements:
                if meas.type == reaction_pb2.ProductMeasurement.ProductMeasurementType.YIELD:
                    data['yield'] = meas.percentage.value
                    break
    
    return data

# Extract all reactions
reaction_data = [extract_reaction_data(rxn) for rxn in tqdm(data.reactions)]
df_clean = pd.DataFrame(reaction_data)

print(f"Extracted {len(df_clean)} reactions")
print(f"Columns: {list(df_clean.columns)}")
df_clean.head()

In [ ]:
# Check for missing values
print("Missing values:")
print(df_clean.isnull().sum())

# Drop rows without yield
df_clean = df_clean.dropna(subset=['yield'])
print(f"\nReactions with yield: {len(df_clean)}")

In [ ]:
# Yield distribution
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(df_clean['yield'], bins=50, color='#3498db', edgecolor='black', alpha=0.7)
ax.set_xlabel('Yield (%)', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Yield Distribution - Ahneman Dataset', fontsize=14, fontweight='bold')
ax.axvline(x=df_clean['yield'].mean(), color='red', linestyle='--', label=f"Mean: {df_clean['yield'].mean():.1f}%")
ax.axvline(x=df_clean['yield'].median(), color='green', linestyle='--', label=f"Median: {df_clean['yield'].median():.1f}%")
ax.legend()
plt.tight_layout()
plt.show()

print(f"Yield statistics:")
print(f"  Min:    {df_clean['yield'].min():.1f}%")
print(f"  Max:    {df_clean['yield'].max():.1f}%")
print(f"  Mean:   {df_clean['yield'].mean():.1f}%")
print(f"  Median: {df_clean['yield'].median():.1f}%")
print(f"  Std:    {df_clean['yield'].std():.1f}%")

## 3. Feature Engineering: One-Hot Encoding

Convert each unique SMILES to a binary feature (one-hot encoding)

In [ ]:
# Identify input columns (everything except yield)
input_cols = [col for col in df_clean.columns if col != 'yield']

print("Input columns for modeling:")
for col in input_cols:
    n_unique = df_clean[col].nunique()
    print(f"  {col}: {n_unique} unique values")

In [ ]:
# Create one-hot encoding
# Use short prefixes for cleaner feature names
prefix_map = {}
for i, col in enumerate(input_cols):
    # Create short prefix
    prefix_map[col] = col[:10]

ohe_df = pd.get_dummies(df_clean[input_cols], prefix=[prefix_map[c] for c in input_cols])

# Add normalized yield (0-1)
ohe_df['yield'] = df_clean['yield'].values / 100

print(f"One-hot encoded shape: {ohe_df.shape}")
print(f"Features: {ohe_df.shape[1] - 1}")
ohe_df.head()

## 4. Prepare Train/Val/Test Split

In [ ]:
# Create numpy arrays
X = ohe_df.drop(columns=['yield']).values
y = ohe_df['yield'].values

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Split: 60% train, 10% validation, 30% test
_X_train, X_test, _y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(_X_train, _y_train, test_size=0.1/0.7, random_state=42)

print(f"\nTrain: {X_train.shape[0]}")
print(f"Val:   {X_val.shape[0]}")
print(f"Test:  {X_test.shape[0]}")
print(f"Total: {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]}")

## 5. Train Models

In [ ]:
# First, try sklearn models as baseline
print("Training sklearn models...\n")

sklearn_models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
}

sklearn_results = {}

for name, model in sklearn_models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    
    # Predict on test set
    y_pred = model.predict(X_test)
    
    # Metrics (convert back to percentage)
    rmse = np.sqrt(mean_squared_error(y_test * 100, y_pred * 100))
    mae = mean_absolute_error(y_test * 100, y_pred * 100)
    r2 = r2_score(y_test, y_pred)
    
    sklearn_results[name] = {'model': model, 'rmse': rmse, 'mae': mae, 'r2': r2, 'y_pred': y_pred}
    print(f"  RMSE: {rmse:.2f}%, MAE: {mae:.2f}%, R²: {r2:.4f}\n")

In [ ]:
# Neural Network (following ORD creator's approach)
print("Training Neural Network...\n")

# Create TensorFlow datasets
batch_size = 100
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32), y_train.astype(np.float32))).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val.astype(np.float32), y_val.astype(np.float32))).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.astype(np.float32), y_test.astype(np.float32))).batch(batch_size)

# Build model (same architecture as ORD creators)
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(50, activation='sigmoid'),
    keras.layers.Dense(7, activation='sigmoid'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(0.005),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.RootMeanSquaredError()]
)

model.summary()

In [ ]:
# Train with early stopping based on validation loss
epochs = 300

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=0
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=30,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, early_stopping],
    verbose=1
)

In [ ]:
# Plot training history
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history.history['loss'], label='Training Loss', color='blue')
ax.plot(history.history['val_loss'], label='Validation Loss', color='green')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss (MSE)')
ax.set_title('Training History', fontsize=14, fontweight='bold')
ax.legend()
ax.set_ylim(0, max(history.history['loss'][5:]) * 1.5)  # Zoom in after initial epochs
plt.tight_layout()
plt.show()

best_epoch = np.argmin(history.history['val_loss'])
print(f"Best epoch: {best_epoch}")

In [ ]:
# Evaluate Neural Network on test set
y_pred_nn = model.predict(test_dataset).flatten()

# Metrics
rmse_nn = np.sqrt(mean_squared_error(y_test * 100, y_pred_nn * 100))
mae_nn = mean_absolute_error(y_test * 100, y_pred_nn * 100)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"Neural Network Test Results:")
print(f"  RMSE: {rmse_nn:.2f}%")
print(f"  MAE:  {mae_nn:.2f}%")
print(f"  R²:   {r2_nn:.4f}")

## 6. Model Comparison

In [ ]:
# Compare all models
print("="*60)
print("MODEL COMPARISON")
print("="*60)

all_results = {
    **sklearn_results,
    'Neural Network': {'rmse': rmse_nn, 'mae': mae_nn, 'r2': r2_nn, 'y_pred': y_pred_nn}
}

comparison_df = pd.DataFrame({
    'Model': list(all_results.keys()),
    'RMSE (%)': [all_results[m]['rmse'] for m in all_results],
    'MAE (%)': [all_results[m]['mae'] for m in all_results],
    'R²': [all_results[m]['r2'] for m in all_results]
}).sort_values('RMSE (%)')

print(comparison_df.to_string(index=False))

# Baseline comparison
mean_baseline_rmse = np.sqrt(mean_squared_error(y_test * 100, np.full_like(y_test, y_train.mean()) * 100))
print(f"\nBaseline (predict mean): RMSE = {mean_baseline_rmse:.2f}%")

In [ ]:
# Visualize best model predictions
best_model_name = comparison_df.iloc[0]['Model']
y_pred_best = all_results[best_model_name]['y_pred']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Actual vs Predicted
ax = axes[0]
ax.scatter(y_test * 100, y_pred_best * 100, alpha=0.5, s=10, c='#3498db')
ax.plot([0, 100], [0, 100], 'r--', label='Perfect prediction')
ax.set_xlabel('Actual Yield (%)', fontsize=12)
ax.set_ylabel('Predicted Yield (%)', fontsize=12)
ax.set_title(f'{best_model_name}: Actual vs Predicted', fontsize=14, fontweight='bold')
ax.set_xlim(-5, 105)
ax.set_ylim(-5, 105)
ax.legend()

# Residual distribution
ax = axes[1]
residuals = (y_test - y_pred_best) * 100
ax.hist(residuals, bins=50, color='#3498db', edgecolor='black', alpha=0.7)
ax.set_xlabel('Residual (Actual - Predicted) %', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Residual Distribution', fontsize=14, fontweight='bold')
ax.axvline(x=0, color='red', linestyle='--')

plt.tight_layout()
plt.show()

# Prediction accuracy
errors = np.abs(y_test - y_pred_best) * 100
print(f"\nPrediction accuracy ({best_model_name}):")
print(f"  Within ±5%:  {100*np.mean(errors <= 5):.1f}% of predictions")
print(f"  Within ±10%: {100*np.mean(errors <= 10):.1f}% of predictions")
print(f"  Within ±20%: {100*np.mean(errors <= 20):.1f}% of predictions")

## 7. Feature Importance (Random Forest)

In [ ]:
# Get feature importance from Random Forest
rf = sklearn_results['Random Forest']['model']
feature_names = ohe_df.drop(columns=['yield']).columns
importances = rf.feature_importances_

# Get top 20
indices = np.argsort(importances)[::-1][:20]

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(range(20), importances[indices], color='#3498db', edgecolor='black')
ax.set_xticks(range(20))
ax.set_xticklabels([feature_names[i][:20] for i in indices], rotation=45, ha='right')
ax.set_ylabel('Importance')
ax.set_title('Top 20 Most Important Features for Yield Prediction', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Summary

### What we learned:
1. **This IS real ML** - predicting yield from structure/conditions has no simple rules
2. **One-hot encoding works** - treats each unique molecule as a categorical feature
3. **Model comparison** - see which approach works best for this dataset

### Limitations:
- One-hot encoding doesn't generalize to new molecules
- For new substrates, would need molecular fingerprints or other representations

### Next steps:
- Test on external dataset to check generalization
- Try molecular fingerprints instead of one-hot encoding
- Add more condition features (temperature, time, etc.)